<a href="https://colab.research.google.com/github/binlee52/Individual-Research-1/blob/master/Script_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pip==18.0

In [2]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Writing setup.sh


In [0]:
!git clone https://github.com/NVIDIA/apex
!cd apex
!pip install -v --no-cache-dir apex

In [0]:
!sh setup.sh

In [0]:
!git clone https://github.com/NVIDIA/apex

In [0]:
!cd apex

In [0]:
!pip install -v --no-cache-dir apex

In [0]:
pip install transformers==2.10.0

In [0]:
pip install simpletransformers

In [0]:
from transformers import *

In [0]:
import tensorflow as tf
import tensorflow_datasets
import pandas as pd

In [0]:
# Google drive 연결
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
checkpoint_path = '/content/drive/My Drive/2020년 1학기/개별연구/checkpoint/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [0]:
train_link = '/content/drive/My Drive/Colab Notebooks/tagged_script/tt0184858.csv'
train_df = pd.read_csv(train_link, dtype = {'sentence': str, 'label': int})
train_df.columns = ['text', 'labels']

In [119]:
train_df[:10]

,text,labels
0,"""REINDEER GAMES""",0
1,Screenplay by,0
2,Ehren Kruger,0
3,FADE IN:,0
4,FIGURE IN A SANTA CLAUS SUIT lies face-down in...,1
5,One of the body's red-sleeved arms is twisted ...,1
6,The white snow beneath the figure is spreading...,1
7,REVEAL EXT.,5
8,SNOWY ROAD,5
9,NIGHT,6


In [18]:
del eval_df
eval_link = '/content/drive/My Drive/2020년 1학기/개별연구/eval_data.csv'
eval_df = pd.read_csv(eval_link, dtype = {'sentence': str, 'label': int})
eval_df.columns = ['text', 'labels']
eval_df[:10]

,text,labels
0,"In the darkness, we hear the solitary sound of...",1
1,NARRATOR,2
2,In between the years when the oceans drank Atl...,3
3,"Hither came Conan, the Cimmerian: a thief, a s...",3
4,"Muffled sounds, as if underwater, echo:",1
5,"CLANGING swords, the guttural CRIES of combat.",1
6,UNBORN BABY,2
7,"Eyes closed, floating at peace within red glow...",1
8,"Suddenly, a flash of steel, as a sword pierces...",1
9,"As the sword is ripped out, light streams in f...",1


In [0]:
from simpletransformers.classification import ClassificationModel

In [0]:
checkpoint_path = '/content/drive/My Drive/2020년 1학기/개별연구/checkpoint/'
bestmodel_path = '/content/drive/My Drive/2020년 1학기/개별연구/best_model/'

In [0]:
params = {
          'num_train_epochs': 10,
          'reprocess_input_data': True,
          'overwrite_output_dir': True,
          'output_dir': checkpoint_path,
          'learning_rate':4e-5,
          'save_model_every_epoch': True,
          }

In [0]:
# Create a TransformerModel with modified attributes
model = ClassificationModel('bert', 'bert-base-cased', num_labels=8, args=params)

In [0]:
?model.train_model()

In [120]:
# Train the model
model.train_model(train_df)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.878687

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.570283Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.061117Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.255985Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.000162

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

Running loss: 0.271073Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.000102


Running loss: 0.000028


Running loss: 0.000513


Running loss: 0.000056


Running loss: 0.000023



In [0]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

In [123]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=accuracy_score)

In [124]:
print(result, model_outputs, wrong_predictions)

{'mcc': 0.8023935179650683, 'acc': 0.8540620384047267, 'eval_loss': 1.3221581266173776} [[-2.0527344  10.90625    -2.5390625  ... -1.9160156  -2.3085938
  -3.2636719 ]
 [-1.8164062  -2.2402344   2.3300781  ...  7.4921875  -1.7304688
  -5.828125  ]
 [-2.2246094  10.71875    -2.6816406  ... -1.9423828  -2.1542969
  -3.6621094 ]
 ...
 [-2.7207031  -0.94140625 -1.8779297  ... -2.1230469  -2.0371094
  -3.2265625 ]
 [-2.5253906   0.01296997 -2.0644531  ... -2.3242188  -2.3691406
  -3.5566406 ]
 [ 5.6523438   1.1083984  -1.3544922  ... -0.54248047 -2.2636719
  -3.5058594 ]] [<simpletransformers.classification.classification_utils.InputExample object at 0x7fc27db65a20>, <simpletransformers.classification.classification_utils.InputExample object at 0x7fc27db65a58>, <simpletransformers.classification.classification_utils.InputExample object at 0x7fc27db65a90>, <simpletransformers.classification.classification_utils.InputExample object at 0x7fc27db65b38>, <simpletransformers.classification.classi

In [125]:
predictions, raw_outputs = model.predict(['JOHN'])
predictions, raw_outputs

(array([2]),
 array([[-3.2519531, -1.7128906,  6.8203125, -1.2646484, -6.3320312,
          3.3691406, -2.8769531, -6.2851562]], dtype=float32))

In [126]:
predictions, raw_outputs = model.predict(['No thanks, I\'m not staying long.'])
predictions, raw_outputs

(array([3]),
 array([[-2.6875   , -1.1816406, -1.8173828, 10.9453125, -3.6191406,
         -1.9814453, -2.0214844, -3.2265625]], dtype=float32))

In [127]:
predictions, raw_outputs = model.predict(['FATHER GARRET and Angela have retreated to another area for privacy.'])
predictions, raw_outputs

(array([1]),
 array([[-2.1210938, 10.8984375, -2.3359375, -1.4433594, -3.4707031,
         -1.9941406, -2.578125 , -3.3886719]], dtype=float32))

In [128]:
predictions, raw_outputs = model.predict(['ANGELA'])
predictions, raw_outputs

(array([2]),
 array([[-2.9863281, -2.9082031,  8.1171875, -1.0205078, -5.546875 ,
          1.7285156, -2.4335938, -5.53125  ]], dtype=float32))

In [129]:
predictions, raw_outputs = model.predict(['THEOLOGICAL SOCIETY'])
predictions, raw_outputs

(array([5]),
 array([[-1.578125 , -1.9082031, -0.7680664, -2.1015625, -3.8007812,
          9.2421875, -1.3359375, -3.3574219]], dtype=float32))

In [130]:
predictions, raw_outputs = model.predict(['FADE IN:'])
predictions, raw_outputs

(array([0]),
 array([[ 9.0234375, -1.0537109, -0.7089844, -2.3183594, -1.0449219,
         -0.5839844, -0.5019531, -0.7944336]], dtype=float32))

In [136]:
predictions, raw_outputs = model.predict(['FADE IN'])
predictions, raw_outputs

(array([5]),
 array([[-1.3857422 , -1.7646484 ,  0.07928467, -2.5195312 , -5.2578125 ,
          8.84375   , -1.2451172 , -4.7382812 ]], dtype=float32))

In [137]:
predictions, raw_outputs = model.predict(['LATER'])
predictions, raw_outputs

(array([6]),
 array([[-0.62060547, -1.7910156 , -1.9980469 , -1.6005859 , -1.7705078 ,
         -1.0644531 ,  9.3671875 , -1.5527344 ]], dtype=float32))

In [139]:
predictions, raw_outputs = model.predict(['Our view RISES OVER him and gliding across the barren landscape are the SHADOW of WINGS.'])
predictions, raw_outputs

(array([1]),
 array([[-1.9990234, 10.8984375, -2.40625  , -1.6035156, -3.6152344,
         -1.7773438, -2.4804688, -3.484375 ]], dtype=float32))

In [140]:
predictions, raw_outputs = model.predict(['MATCH CUT TO:'])
predictions, raw_outputs

(array([0]),
 array([[ 9.3046875 , -1.6318359 , -0.7080078 , -2.6796875 , -0.57421875,
         -0.5961914 ,  0.14453125,  0.09338379]], dtype=float32))

In [141]:
predictions, raw_outputs = model.predict(['That\'s called pain.'])
predictions, raw_outputs

(array([3]),
 array([[-2.6914062, -0.4567871, -1.9316406, 10.953125 , -3.8808594,
         -2.0761719, -2.3476562, -3.4003906]], dtype=float32))

In [147]:
predictions, raw_outputs = model.predict(['PALACE'])
predictions, raw_outputs

(array([5]),
 array([[-1.4423828 , -2.4960938 ,  0.01489258, -2.4980469 , -4.6914062 ,
          8.9375    , -0.77490234, -4.21875   ]], dtype=float32))